# FVCOM4 2D plotting using datashader and DynamicMap
**Author: Jun Sasaki, coded on June 30, 2018, updated on October 5, 2022** <br>
- dev_FVCOM_2D_hv.ipynbを発展させ，datashaderでカラーマップを描き，meshはholoviewsのtrimesh.edgepathsで描く
- 時間ステップを含む平面2次元と空間3次元データを切り出し，カラーマップを描けるようにした
- 項目，時刻，σ層等を与えて，一気に描画するClass methodを作成した
- geoviews対応が課題
### The following warnings may appear.
- WARNING:Fiona:GDAL data files not located, GDAL_DATA not set
- WARNING:Fiona:PROJ data files not located, PROJ_LIB not set

## Install firefox on WSL2 Ubuntu 20.04LTS
```
$ sudo apt update
$ sudo apt install firefox
```
## Install packages on Miniconda Python confirmed on Oct 25, 2021
```
$ conda config --add channels conda-forge
$ conda install pandas geopandas matplotlib netcdf4 xarray
$ conda install cartopy
$ conda install hvplot
$ conda install pygmt
$ conda install jupyterlab
$ conda install geoviews
$ conda install selenium
$ conda install firefox geckodriver
```


In [ ]:
import sys
import os
from functools import partial, partialmethod
import numpy as np
import pandas as pd
import pyproj
### import xarray as xr
import netCDF4
### import matplotlib.pyplot as plt
import matplotlib.tri as tri
import holoviews as hv
from holoviews import opts
#import datashader as ds
import datashader.utils as du
#import datashader.transfer_functions as tf
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize, spread, aggregate, regrid
from holoviews.operation import decimate
import holoviews.plotting.mpl
#from holoext.xbokeh import Mod
import geoviews as gv
from bokeh import models
import subprocess
from bokeh.io import export_png
#from holoviews.plotting.bokeh.element import (line_properties, fill_properties, text_properties)
hv.extension('bokeh', 'matplotlib')

## 平面直角座標と地理座標の変換を行う関数定義

In [ ]:
def proj_trans(df=None, col_x='Longitude', col_y='Latitude', epsg0="2451", epsg1="4612"):
    '''DataFrame dfの最初の2つのカラムの(x,y)をepsg0からepsg1座標系に変換し，
       (x, y)のカラム名を(col_x, col_y)に変更したDataFrameを返す'''
    EPSG0 = pyproj.Proj("+init=EPSG:" + epsg0)
    EPSG1 = pyproj.Proj("+init=EPSG:" + epsg1)
    x1,y1 = pyproj.transform(EPSG0, EPSG1, df.iloc[:,0].values, df.iloc[:,1].values)
    ### print(len(x1))
    ### print(len(df))
    df[df.columns[0]]=x1
    df[df.columns[1]]=y1
    df.rename(columns = {df.columns[0]:col_x, df.columns[1]:col_y}, inplace=True)
    return df

## xlimとylimを設定するとInteractiveに解像度が変化しなくなる
- xlimとylimを固定して与える場合と与えない場合を分けて作っておく必要がある．前者は静的に扱い，後者は動的に扱うことを前提とする
- Mod()は複数のインスタンスを\*で重ねると効かない場合がある．例えば，軸名称が元に戻ってしまう．classの段階では最小限の設定で作り，装飾は後で行うのがよいだろう
- 一つの時間ステップtimeを引数として，基本的な描画を担う，関数time_zeta(time)を定義します．この段階でDatashaderを適用してはいけません．Datashaderはdynamic mapの一つで，後でDynamicMapを適用すると，DynamicMapのネストはサポートされていないというエラーとなるためです．さらに，この段階では.redim.range()を適用せず，描画段階で適用することで，フレキシビリティを確保します．
- 2d mapにおいて，cmapはrasterizeの段階で適用する必要がありそう．opts.Imageやopts.TriMeshでは適用できなかった．datashaderのオプションの与え方について要検討

In [ ]:
class Fvcom2D:
    '''Postprocessing for FVCOM 4 netcdf output in its original coordinates'''
    def __init__(self, ncfile_path='./output/tokyo_0001.nc', obcfile_path='./inputfile/tokyo_obc.dat', m_to_km=True, offset=False):
        '''m_to_km=True if converting x and y axis units from m to km.
           offset=True if the left-bottom corner is set to the origin.'''
        self.fvcom_nc=ncfile_path
        if os.path.isfile(self.fvcom_nc):
            self.FVCOM = netCDF4.Dataset(self.fvcom_nc, 'r')
        else:
            print("ERROR: Input netcdf file does not exit.")
            sys.exit()
        self.variables = self.FVCOM.variables
        print(self.variables.keys())
        self.fobc=obcfile_path # = None if not exist
        if os.path.isfile(self.fobc):
            df = pd.read_csv(self.fobc, header=None, skiprows=1, delim_whitespace=True)
            ### -1 because index in FVCOM starts from 1 while from 0 in Python
            self.node_bc = df.iloc[:,1].values - 1
            print('Open boundary nodes = ', self.node_bc)
        self.x, self.y = self.variables['x'][:], self.variables['y'][:] ### at node
        self.xc, self.yc = self.variables['xc'][:], self.variables['yc'][:]   ### at cell center
        ### Convert axis units from m to km
        if m_to_km:
            self.x /= 1000.0; self.y /= 1000.0; self.xc /= 1000.0; self.yc /= 1000.0
        if offset:
            xmin = self.x.min(); ymin = self.y.min()
            self.x -= xmin; self.y -= ymin; self.xc -= xmin; self.yc -= ymin
        self.siglay, self.siglev = self.variables['siglay'][:], self.variables['siglev'][:]
        # self.siglay_center, self.siglev_center = self.variables['siglay_center'][:], self.variables['siglev_center'][:]
        self.iint = self.variables['iint'][:]
        self.time = self.variables['time'][:]
        self.Itime = self.variables['Itime'][:]
        self.Itime2 = self.variables['Itime2'][:]
        self.z = self.variables['h'][:]
        ### verts[(x0,y0,h0),[x1,y1,h1],...,[xn,yn,hn]]
        verts = [(xi, yi, hi) for xi, yi, hi in zip(self.x.data, self.y.data, self.z.data)]
        self.verts = pd.DataFrame(verts, columns=['x', 'y', 'z'])
        ### Get connectivity array nv (3 node numbers of element i)
        nv = self.variables['nv'][:].T - 1
        self.triang = tri.Triangulation(self.x, self.y, triangles=nv)
        ### FVCOMでは時計回りに定義されているので，matplotlib.triの仕様である反時計回りに変更する．node番号自体は不変．
        self.nv=nv[:,::-1]
        self.tris = pd.DataFrame(self.nv, columns=['v0', 'v1', 'v2'])
        self.mesh = du.mesh(self.verts, self.tris) 
        self.trimesh = hv.TriMesh((self.tris, self.verts))
        #self.edgepaths = self.trimesh.edgepaths
        ### Element index of 3 neighbors of element i 各セルには一般に3つのセルが隣接します（境界セルはこの限りではない）
        self.nbe = np.array([[self.nv[n, j], self.nv[n, (j+2)%3]]\
                             for n in range(len(self.triang.neighbors)) for j in range(3) if self.triang.neighbors[n,j] == -1])

    @property
    def timestamp(self):
        '''Create time stamp list in ["D HH:MM:SS"]
           様々なtimestamp形式に対応させるには，Classとして独立させるとよいかもしれない．
        '''
        dayi = self.Itime
        hourf = self.Itime2 / 3600000
        houri = hourf.astype('int32')
        minf = (hourf - houri) * 60
        mini = minf.astype('int32')
        secf = (minf - mini) * 60
        seci = secf.astype('int32')
        return [f"{d} {h:02}:{m:02}:{s:02}" for d, h, m, s in zip(dayi, houri, mini, seci)]

    def plot_mesh(self, x_range=None, y_range=None, **kwargs):
        ''' width=300, height=300, line_color="blue", line_width=0.1', x_range=None, y_range=None '''
        width=kwargs.get('width', 300)
        height=kwargs.get('height', 300)
        line_color=kwargs.get('line_color', 'blue')
        line_width=kwargs.get('line_width', 0.1)
        p = self.trimesh.edgepaths.options(width=width, height=height, line_width=line_width, line_color=line_color)
        if x_range == None and y_range == None:
            return p
        elif y_range == None:
            return p.redim.range(x=x_range)
        elif x_range == None:
            return p.redim.range(y=y_range)
        else:
            return p.redim.range(x=x_range, y=y_range)
#        return self.trimesh.edgepaths.options(width=width, height=height, line_width=line_width, line_color=line_color)
    
    def plot_coastline(self, x_range=None, y_range=None, **kwargs):
        ''' width=300, height=300, color='red', line_width=0.5
            海岸線plot 時間がかかるので，interactive plottingでは非推奨
        '''
        ### paths = [[(x0s, y0s), (x0e, y0e)], [(x1s, y1s), (x1e, y1e)], ..., [(xns, yns), (xne, yne)]]
        width=kwargs.get('width', 300)
        height=kwargs.get('height', 300)
        color=kwargs.get('color', 'red')
        line_width=kwargs.get('line_width', 0.5)
        paths = [[(self.x[self.nbe[m,:]][0], self.y[self.nbe[m,:]][0]), (self.x[self.nbe[m,:]][1], self.y[self.nbe[m,:]][1])] \
                 for m in range(len(self.nbe))]
        p = hv.Path(paths).options(width=width, height=height, color=color, line_width=line_width)
        if x_range == None and y_range == None:
            return p
        elif y_range == None:
            return p.redim.range(x=x_range)
        elif x_range == None:
            return p.redim.range(y=y_range)
        else:
            return p.redim.range(x=x_range, y=y_range)
        
#        return hv.Path(paths).options(width=width, height=height, color=color, line_width=line_width)

    def get_val(self, item, time=None, sigma=None):
        ### Error checks
        if not item in self.variables.keys():
            print("Error: This item does not exit.")
            sys.exit()
        val = self.FVCOM.variables[item]

        if len(val.shape) == 1 : ### 1-D item
            scalar = val
        elif len(val.shape) >= 2: ### 2-D or 3-D item
            if time < 0 or time > val.shape[0]:
                print("ERROR: Time index = ", time, "is out of range.")
                print("Time index should be between ", 0, "and ", val.shape[0]-1, " .")
                sys.exit()
            if len(val.shape) == 2: ### 2-D item
                scalar = val[time]
            elif len(val.shape) == 3: ### 3-D item
                if sigma < 0 or sigma > val.shape[1]:
                    print("ERROR: Sigma layer number = ", sigma, "is out of range.")
                    print("Sigma layer number should be between ", 0, " and ", val.shape[1]-1, " .")
                    sys.exit()
                else:
                    scalar = val[time][sigma]
            else:
                print("ERROR: the shape is incorrect.")
                sys.exit()
        verts = [(xi, yi, hi) for xi, yi, hi in zip(self.x.data, self.y.data, scalar)]
        return pd.DataFrame(verts, columns=['x', 'y', item])

    def __plot_2dh(self, item, time, sigma):
        if len(self.variables[item].shape) == 1:   ### 1-D item (ex: depth) 
            item = self.get_val(item)
        elif len(self.variables[item].shape) == 2: ### 2-D item (ex: zeta)
            item = self.get_val(item, time=time)
        elif len(self.variables[item].shape) == 3: ### 3-D item (ex: salainity)
            item = self.get_val(item, time=time, sigma=sigma)
        else:
            print('ERROR: The shape of the item is incorrect in __plot_2dh')
            sys.exit()
        return hv.TriMesh((self.tris, hv.Points(item)))

    def dmap(self, item, x_range=None, y_range=None, z_range=None, timestamp_location=None):
        '''Interactive plotting using DynamicMap
           内部関数を用意すること，partialの引数の最後をtimeとsigmaにするのがポイント
        '''
        def item3d_plot(item, timestamp_location, sigma, time):  ###  Should be internal function
            '''self: fvcom instance, siglay: sigma layer
               partialの引数とするため，内部関数とする．そうでないとうまくいかなかった
            '''
            item = self.get_val(item, time=time, sigma=sigma)
            timestamp_txt = self.plot_timestamp(time, timestamp_location)
            return hv.TriMesh((self.tris, hv.Points(item))) * timestamp_txt

        def item2d_plot(item, timestamp_location, time):
            '''self: fvcom instance'''
            item = self.get_val(item, time=time)
            timestamp_txt = self.plot_timestamp(time, timestamp_location)
            return hv.TriMesh((self.tris, hv.Points(item))) * timestamp_txt        
        
        if len(self.variables[item].shape) == 1:   ### 1-D item (ex: depth) 
            print('ERROR: 1-D item is not supported in dmap in Class FVCOM_2D')
            sys.exit()
        elif len(self.variables[item].shape) == 2: ### 2-D item (ex: zeta)
            p = partial(item2d_plot, item, timestamp_location)
            dmap = hv.DynamicMap(p, kdims=['time']).redim.values(time=range(len(self.time)))
        elif len(self.variables[item].shape) == 3: ### 3-D item (ex: salainity)
            p = partial(item3d_plot, item, timestamp_location)
            dmap = hv.DynamicMap(p, kdims=['sigma', 'time']).redim.values(sigma=range(len(self.siglay))).redim.values(time=range(len(self.time)))
        else:
            print('ERROR: The shape of the item is incorrect in dmap in Class FVCOM_2D')
            sys.exit()
        #x_range=kwargs.get('x_range', (self.x.min(), self.x.max()))
        #y_range=kwargs.get('y_range', (self.y.min(), self.y.max()))
        ### timestamp_txt = self.plot_timestamp(time, timestamp_location) ### does not work

        if x_range is None or x_range == False:
            x_range = (self.x.min(), self.x.max())
        if y_range is None or y_range == False:
            y_range = (self.y.min(), self.y.max())
        if z_range is None or z_range == False:
            return rasterize(dmap, x_range=x_range, y_range=y_range)
        else:
            return rasterize(dmap.redim.range(**{item : z_range}), x_range=x_range, y_range=y_range)

    def splot_2dh(self, item='h', time=0, sigma=0, x_range=None, y_range=None, z_range=None, \
                  timestamp_location=None, **kwargs):
        '''
        Static horizontal 2D plot (one sigma and time frame)
        静的な平面2次元プロット．InteractiveにはDynamicMapを用いたdmap()を使用する
        '''
        if x_range is None or x_range == False:
            x_range = (self.x.min(), self.x.max())
        if y_range is None or y_range == False:
            y_range = (self.y.min(), self.y.max())
        #x_range=kwargs.get('x_range', (self.x.min(), self.x.max()))
        #y_range=kwargs.get('y_range', (self.y.min(), self.y.max()))
        #z_range=kwargs.get('z_range', (self.variables[item].min(), self.variables[item].max()))
        width = kwargs.get('width', 500)
        height = kwargs.get('height', 500)
        bgcolor = kwargs.get('bgcolor', 'lightgrey')
        show_grid = kwargs.get('show_grid', False)
        xlabel = kwargs.get('xlabel', 'X (km)')
        ylabel = kwargs.get('ylabel', 'Y (km)')
        colorbar = kwargs.get('colorbar', True)
        colorbar_cmap = kwargs.get('colorbar_cmap', 'Reds') # Blues
        colorbar_n = kwargs.get('colorbar_n', 256)
        ### colorbar_opts
        colorbar_title = kwargs.get('colorbar_title', '')
        title_text_font_size = kwargs.get('title_text_font_size', '16pt')
        title_text_baseline = kwargs.get('title_text_baseline', 'bottom')
        title_standoff = kwargs.get('title_standoff', 14)
        title_text_font_style = kwargs.get('title_text_font_style', 'normal')
        major_label_text_font_size = kwargs.get('major_label_text_font_size', '14pt')
        major_label_text_align = kwargs.get('major_label_text_align', "left")
        label_standoff = kwargs.get('label_standoff', 4)
        colorbar_width = kwargs.get('colorbar_width', 20)

        toolbar_location = kwargs.get('toolbar_location', 'top') ### None, 'top', 'right', 'left', 'bottom'
        label_scaler = kwargs.get('label_scaler', 1.2)
        label_alpha = kwargs.get('label_alpha', 1)
        axis_line_alpha = kwargs.get('axis_line_alpha', 1)  ### Opacity
        major_tick_alpha = kwargs.get('major_tick_alpha', 1)
        minor_tick_alpha = kwargs.get('minor_tick_alpha', 1)
        
        colorbar_opts={"title":colorbar_title, "title_text_font_size":title_text_font_size, \
                       "title_text_baseline":title_text_baseline, "title_standoff":title_standoff, \
                       "title_text_font_style":title_text_font_style, \
                       "major_label_text_font_size":major_label_text_font_size, \
                       "major_label_text_align":major_label_text_align, \
                       "label_standoff":label_standoff, "width":colorbar_width}
        plot_opts = dict(colorbar_opts = colorbar_opts)

        p = self.__plot_2dh(item=item, time=time, sigma=sigma)
        #p_options = opts.TriMesh(cmap='viridis')
        #p = p.opts(p_options)
        if z_range is None or z_range == False:
            p = rasterize(p, x_range=x_range, y_range=y_range)
        else:
            p = rasterize(p.redim.range(**{item : z_range}), x_range=x_range, y_range=y_range)

        p = p.opts(cmap="viridis")
        timestamp_txt = self.plot_timestamp(time, timestamp_location)
        # debug jsasaki
        #return p.opts(plot=plot_opts).redim.label(x=xlabel, y=ylabel) * timestamp_txt
        # return p.options(plot=plot_opts) * timestamp_txt
        # return
        # p_options = opts.Image(tools=['hover'])
        #return p.opts(p_options)
        return p * timestamp_txt

    def plot_point(self, coords, color='r', marker='+', size=20):
        p = hv.Points(coords)
        return p.opts(color=color, marker=marker, size=size)
        
    def plot_timestamp(self, time=0, timestamp_location=None):
        '''timestamp_location: tuple (x, y)
           Currently setting the location using graphic space unsupported in Holoviews
           スクリーン座標系で位置が設定できるとよいが，現在はHoloviewsの仕様で不可
           timestamp_location=Noneのときは空文字を出力することで汎用化した
        '''
        if timestamp_location == None:
            return hv.Text(0, 0, "")
        else:
            timestamp = self.timestamp[time]
            return hv.Text(timestamp_location[0], timestamp_location[1], timestamp)

    def make_frame(self, item=None, title=None, time=None, sigma=None, plot_mesh=True, plot_coastline=True, \
                   x_range=None, y_range=None, z_range=None, toolbar_location=None, timestamp_location=None):
        '''Make a frame at one time step [and one sigma layer (only for 3D)]
           1つのframe出力．アニメーション作成に有効
        '''
        ptri = self.splot_2dh(item='salinity',time=time, sigma=sigma, colorbar_title=title, toolbar_location=None, \
                              x_range=x_range, y_range=y_range, z_range=z_range)
        pmesh = self.plot_mesh(x_range=x_range, y_range=y_range)
        pcoastline = self.plot_coastline(x_range=x_range, y_range=y_range)
        timestamp_txt = self.plot_timestamp(time, timestamp_location)

        if plot_mesh and plot_coastline:
            return ptri * pmesh * pcoastline * timestamp_txt
        elif plot_mesh:
            return ptri * pmesh * timestamp_txt
        elif plot_coastline:
            return ptri * pcoastline * timestamp_txt
        else:
            return ptri * timestamp_txt

class Fvcom2D_trans(Fvcom2D):
    '''Postprocessing for FVCOM 4 with converting coordinates
       Requires function proj_trans()
       (x, y)から(lon, lat)への変換を想定しているが，epsg0とepsg1と座標名を与えることで任意の変換が可能
    '''
    def __init__(self, col_x='Longitude', col_y='Latitude', epsg0="2451", epsg1="4612", \
                 ncfile_path='./output/tokyo_0001.nc', obcfile_path='./inputfile/tokyo_obc.dat'):
        '''m_to_km and offset must be False in super().__init__().'''
        super().__init__( ncfile_path, obcfile_path, m_to_km=False, offset=False)
        self.verts = proj_trans(df=self.verts, col_x=col_x, col_y=col_y, epsg0=epsg0, epsg1=epsg1)
        points = gv.operation.project_points(gv.Points(self.verts, vdims=['z']))
        self.mesh = du.mesh(self.verts, self.tris)
        self.trimesh = hv.TriMesh((self.tris, points))
        self.verts = points

In [ ]:
ncfile = 'tst_0001.nc'
obcfile = 'tst_obc.dat'

In [ ]:
fvcom = Fvcom2D(ncfile_path=ncfile, obcfile_path=obcfile, m_to_km=True, offset=False)

In [ ]:
#fvcom.FVCOM.variables['zeta'][0]
#fvcom.variables['zeta'][0]
#len(fvcom.variables['zeta'].shape)
fvcom.variables['zeta'][0].min()

In [ ]:
#fvcom.variables['zeta'][1].max()
np.where(fvcom.variables['zeta'][24] > 10)

## Quick plotting at one time (and sigma) frame using splot_2dh instance method
- **注意** `z`が一様分布のときは`z_range()`を指定しないと描画されない．
- **Caution:** No plotting when `z_range=None` and `z` is uniform. 
- `timestamp_location=` tuple `(x, y)` or `None`
- `toolbar_location=` `None` or `'top'` or `'east'` or `'west'`
- To overlay mesh, append ` * fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)`
- To overlay coastline, append ` * fvcom.plot_coastline(color='red', line_width=0.5, x_range=x_range, y_range=y_range)`
- **Caution:** Plotting coastline takes considerable time.

In [ ]:
#fvcom.splot_2dh(item='zeta', time=50, x_range=(0,22), y_range=(0, 33), z_range=(0,0.2) )
#fvcom.splot_2dh(item='zeta', time=50, x_range=None, y_range=None, z_range=None )

p = fvcom.splot_2dh(item='salinity',time=10, sigma=0, x_range= None, y_range=None, z_range=(0, 35), \
                toolbar_location='east', timestamp_location=(270, 148), colorbar_title='Sal')
coastline = fvcom.plot_coastline(color='red', line_width=0.5, x_range=None, y_range=None)
#p * coastline
p_options = opts.Image(tools=['hover'], width=600, height=250, colorbar=True)
p.opts(p_options) * coastline
p

## Plot mesh

In [ ]:
coords1 = [[fvcom.variables['x'][10].item()/1000, fvcom.variables['y'][10].item()/1000]]
coords2 = [[fvcom.variables['x'][11].item()/1000, fvcom.variables['y'][11].item()/1000]]

In [ ]:
#fvcom.trimesh_edgepaths()
x_range=(370,400)
y_range=(3890, 3950)
mesh = fvcom.plot_mesh(width=500, height=550)
points1 = fvcom.plot_point(coords1, color='red')
points2 = fvcom.plot_point(coords2, color='black')
mesh * points1 * points2
#p.redim.range(x=x_range, y=y_range) * points1 * points2

## Plot coastlines

In [ ]:
'''  ### Included in class FVCOM_2D
### paths = [[(x0s, y0s), (x0e, y0e)], [(x1s, y1s), (x1e, y1e)], ..., [(xns, yns), (xne, yne)]]
paths = [[(fvcom.x[fvcom.nbe[m,:]][0], fvcom.y[fvcom.nbe[m,:]][0]), \
          (fvcom.x[fvcom.nbe[m,:]][1], fvcom.y[fvcom.nbe[m,:]][1])] for m in range(len(fvcom.nbe))]
coastlines=hv.Path(paths)
coastlines
'''

In [ ]:
mesh * fvcom.plot_coastline()

In [ ]:
#fvcom.splot_2dh(item='salinity',time=90, sigma=0, toolbar_location=None, colorbar_title='Sal') * fvcom.plot_mesh() * fvcom.plot_coastline()
#x_range=(0,22); y_range=(0,33)
x_range=None; y_range=None
timestamp_location=(4,3)
fvcom.splot_2dh(item='salinity',time=1, sigma=9, colorbar_title='Sal', x_range=x_range, y_range=y_range,\
                timestamp_location=timestamp_location) \
* fvcom.plot_mesh(x_range=x_range, y_range=y_range) * fvcom.plot_coastline(x_range=x_range, y_range=y_range)

# Make PNG frames
- Make a frame in png using a method of `make_frame()` in class FVCOM_2D

In [ ]:
pngpath = "./png/"
core = "tri_sal_"
for time in range(2):
    outputfile = f"{pngpath}{core}{time:03}"
    p = fvcom.make_frame(item='salinity', title='Sal', time=time, sigma=0, plot_mesh=False, plot_coastline=False, z_range=(10,35),\
                         timestamp_location=(36,60))
    renderer=hv.renderer('bokeh')
    renderer.save(p, outputfile, fmt='png')

# Make APNG animation
- `apngasm64.exe output.png frame*.png 1 4`

In [ ]:
output_png="animation.png"
cmd = "apngasm64.exe" + " " + pngpath + output_png + " " + pngpath + core + "*.png" + " " + "1 1"
print(cmd)
subprocess.call(cmd, shell=True)
web="file:///C:/work/2021/thesis/MiwaTakeshi2021.4-2022.3/PyFVCOM/png/" + output_png
print(web)

# Intaractive Plotting using DynamicMap
- An instance method of `self.dmap()` creates a dynamic map where `item` should be give.
- `x_range`, `y_range` and `z_range` are optional tuples. If not given, the ranges are automatically set as their full ranges.
- For annotating graph, edit the `%%opts` and `Mod()`.
- **Caution:** Cell magic of `%%opts` should be placed at the top in the cell.
- **Caution:** `%%output holomap='scrubber'` should be disabled in case of kdim containing `sigma`.
- `Mod()` is a method in [`HoloExt()`](https://holoext.readthedocs.io/en/latest/index.html). HoloExt source: c:\Users\jsasa\Miniconda3\Lib\site-packages\holoext\bokeh.py
- To overlay mesh, append ` * fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)`
- To overlay coastline, append ` * fvcom.plot_coastline(color='red', line_width=0.5, x_range=x_range, y_range=y_range)`
- **Caution:** Plotting coastline takes considerable time.

In [ ]:
%%opts Image RGB [width=500 height=550 bgcolor="lightgrey"]
%%opts Image RGB [colorbar_opts={"title":"SL (m)","title_text_font_size":"14pt","title_text_baseline":"bottom","title_standoff":12,"title_text_font_style":"normal","major_label_text_font_size":"14pt","major_label_text_align":"left","label_standoff":4,"width":20}]
%%output size=100
#%%output holomap='scrubber' fps=1

item='salinity'
#x_range=(0,22); y_range=(0,33)
z_range=(10,35)
x_range=None; y_range=None
#item='zeta'
#x_range=None; y_range=None
#z_range=(0, 1.2)
timestamp_location=(33, 60)

dmap_ds=fvcom.dmap(item=item, x_range=x_range, y_range=y_range, z_range=z_range, \
                   timestamp_location=timestamp_location)
mesh = fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)
coastline = fvcom.plot_coastline(color='red', line_width=0.5, x_range=x_range, y_range=y_range)

#p=dmap_ds * mesh * coastline ### with mesh and coastline
p=dmap_ds

'''
Mod(autosize=False, ### %%optsでサイズ指定が効かないためFalse
    show_grid=False,  ### Set True or False
    xlabel='X (km)', ylabel='Y (km)',
    colorbar_n=10,  ### Do not set %%opts for colorbat, using Mod() is more flexible.
    colorbar_cmap='Reds',
    #xticks=[0,10,20,30],
    #yticks=[0,10,20,30,40,50,60],
    toolbar_location="top", ### None: hide toolbar, "top", "east", "west"
    label_scaler=1.2,  ### Set font size magnification factor
    label_alpha=0.95,  ### Opacity
    colorbar_tick_alpha=0.95,  ### Opacity
    colorbar_outline_alpha=0.95,  ### Opacity
    axis_line_alpha=0.95,  ### Opacity
    major_tick_alpha=0.95,
    minor_tick_alpha=0.95
    ).apply(p)
'''

# `%%opts`を使わず`Mod()`だけで制御
## Unimportant and just for reference for how to control without `%%opts`
- olorbar_optsが効かない．holoextのコードを直接書き換える，または，customize_bokeh()で対応する
- 当初はファイル出力のため検討したが，ファイル出力はフレーム毎に出力する必要があるため，この方法は不要．

In [ ]:
colorbar_opts={"title":"SL (m)","title_text_font_size":"16pt","title_text_baseline":"bottom","title_standoff":20,\
               "title_text_font_style":"normal","major_label_text_font_size":"14pt","major_label_text_align":"left",\
               "label_standoff":4,"width":20}
plot_opts=dict(width=500, height=550, colorbar=True, colorbar_opts=colorbar_opts)


#colorbar_opts={"title":"SL (m)","title_text_font_size":"14pt","title_text_baseline":"bottom","title_standoff":12,"title_text_font_style":"normal","major_label_text_font_size":"14pt","major_label_text_align":"left","label_standoff":4,"width":20}
#item='salinity'
item='zeta'
#x_range=(0,22); y_range=(0,33)
x_range=None; y_range=None
#z_range=(10,35)
z_range=(0,0.1)

timestamp_location=(1000, 5000)

dmap_ds=fvcom.dmap(item=item, x_range=x_range, y_range=y_range, z_range=z_range, \
                   timestamp_location=timestamp_location)
mesh = fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)
coastline = fvcom.plot_coastline(color='blue', line_width=0.5, x_range=x_range, y_range=y_range)

p = dmap_ds

'''
saved = Mod(width=500, height=550, bgcolor='lightgrey',
            show_grid=False,  ### Set True or False
            xlabel='X (km)', ylabel='Y (km)',
            colorbar_n=10,  ### Do not set %%opts for colorbat, using Mod() is more flexible.
            colorbar_cmap='Reds',
            #colorbar_title="SL (m)",
            label_alpha=0.95, ### title_text_alpha (colorbar_dict)
            #xticks=[0,10,20,30],
            #yticks=[0,10,20,30,40,50,60],
            toolbar_location="east", ### None: hide toolbar, "top", "east", "west"
            label_scaler=1,  ### Set font size magnification factor
            colorbar_tick_alpha=0.95,  ### Opacity
            colorbar_outline_alpha=0.95,  ### Opacity
            axis_line_alpha=0.95,  ### Opacity
            major_tick_alpha=0.95,
            minor_tick_alpha=0.95
           ).apply(p)
'''
#saved.opts(plot=plot_opts).redim.label(x='X (km)', y='Y (km)')
p.opts(plot=plot_opts).redim.label(x='X (km)', y='Y (km)')
#  * coastline
#saved.redim.label(x='X (km)', y='Y (km)') \
#  * fvcom.plot_coastline(color='red', line_width=0.5).redim.range(x=x_range, y=y_range)


# BokehのPrimitive機能を用い，微調整を行う
## Fine adjustment using Bokeh's primitive functions
- `hv.help(hv.Image)`で調整できるオプションを調べられる
- 上記で調整できないオプションもbokehの機能の`finalize_hooks=[customize_bokeh]`で調整できる．`customize_bokeh`は以下のような関数
```Python
def customize_bokeh(plot, element):
        plot.state.axis.axis_line_width = 1
        plot.state.axis.axis_label_text_font_style = 'normal'
```

In [ ]:
hv.help(hv.RGB)

In [ ]:
def customize_bokeh(plot, element):
    '''Enforce adjustiments using bokeh's primitive functions.'''
    #plot.state.outline_line_color = 'black'
    #plot.state.outline_line_width = 2
    plot.state.axis.axis_line_width = 1.5
    plot.state.axis.axis_label_text_font_style = 'normal'
    ### plot.state.x_range = models.Range1d(0,22) ### Do not apply: Dynamic resolution change disabled.
    ### plot.state.y_range = models.Range1d(0,33)

In [ ]:
%%opts Image RGB [width=500 height=550 bgcolor="lightgrey"]
%%opts Image RGB [colorbar=True color_levels=256] (cmap='Reds')
%%opts Image RGB [colorbar_opts={"title":"SL (m)","title_text_font_size":"14pt","title_text_baseline":"bottom","title_standoff":12,"title_text_font_style":"normal","major_label_text_font_size":"14pt","major_label_text_align":"left","label_standoff":4,"width":20}]
%%opts Image RGB [fontsize={"xlabel":"20pt", "ylabel":"20pt","ticks":"20pt","title":"20pt"}]
%%output size=100
%%opts Image RGB [finalize_hooks=[customize_bokeh]]
#%%opts Image [width=500 height=550 bgcolor="lightgrey"]
#%%opts Image [fontsize={"ticks":"14pt", "title":"14pt", "xlabel":"14pt", "ylabel":"14pt"}]
#%%opts Image [colorbar=True color_levels=256 colorbar_opts={"width": 20, "major_label_text_font_size":"12pt"}] (cmap='ocean_r')
#%%opts Image [finalize_hooks=[customize_bokeh]]

item='salinity'
x_range=(0,22); y_range=(0,33)
z_range=(10,35)
xlabel='X (km)'; ylabel='Y (km)'
timestamp_location=(4,3)

dmap_ds=fvcom.dmap(item=item, x_range=x_range, y_range=y_range, z_range=z_range,\
                   timestamp_location=timestamp_location)
mesh = fvcom.plot_mesh(color='blue', line_width=0.1, x_range=x_range, y_range=y_range)
coastline = fvcom.plot_coastline(color='red', line_width=0.5, x_range=x_range, y_range=y_range)

#p = dmap_ds * coastline
p = dmap_ds

p.redim.label(x=xlabel, y=ylabel)